In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
img_size = 380
IMAGE_SIZE = [img_size, img_size]

In [4]:
from google.colab import drive
drive.mount('/thesis')

Mounted at /thesis


In [5]:

train_path='/thesis/MyDrive/thesis/new_clean/train'
test_path= '/thesis/MyDrive/thesis/new_clean/test'
val_path=  '/thesis/MyDrive/thesis/new_clean/test'

In [6]:
x_train=[]
for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_train.append(img_arr)
        
x_test=[]
for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_test.append(img_arr)

x_val=[]
for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_val.append(img_arr)

/thesis/MyDrive/thesis/new_clean/train/Hispa
/thesis/MyDrive/thesis/new_clean/train/Brown_Spot
/thesis/MyDrive/thesis/new_clean/train/Leaf_Blast
/thesis/MyDrive/thesis/new_clean/test/Brown_Spot
/thesis/MyDrive/thesis/new_clean/test/Hispa
/thesis/MyDrive/thesis/new_clean/test/Leaf_Blast
/thesis/MyDrive/thesis/new_clean/test/Brown_Spot
/thesis/MyDrive/thesis/new_clean/test/Hispa
/thesis/MyDrive/thesis/new_clean/test/Leaf_Blast


In [7]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

train_x.shape,test_x.shape,val_x.shape

train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)
test_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)
val_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (img_size, img_size),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (img_size, img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (img_size, img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

training_set.class_indices
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes
train_y.shape,test_y.shape,val_y.shape

Found 441 images belonging to 3 classes.
Found 76 images belonging to 3 classes.
Found 76 images belonging to 3 classes.


((441,), (76,), (76,))

In [10]:
efficientnet = EfficientNetB4(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


71696384/71686520 [==============================] - 2s 0us/step


In [11]:
for layer in efficientnet.layers:
    layer.trainable = False

x = Flatten()(efficientnet.output)

prediction = Dense(3, activation='softmax')(x)


model = Model(inputs=efficientnet.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 380, 380, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 380, 380, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 380, 380, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 381, 381, 3)  0           ['normalization[0][0]']      

In [12]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)



In [ ]:
for x in range(15):
    print("\n\niteration: "+str(x));
    history = model.fit(
      train_x,
      train_y,
      validation_data=(val_x,val_y),
      epochs=20,  
      batch_size=8,shuffle=True)



iteration: 0
Epoch 1/20
56/56 [==============================] - 34s 216ms/step - loss: 25.3234 - accuracy: 0.4036 - val_loss: 14.7974 - val_accuracy: 0.3947
Epoch 2/20
56/56 [==============================] - 8s 135ms/step - loss: 14.6951 - accuracy: 0.3991 - val_loss: 28.7104 - val_accuracy: 0.3947
Epoch 3/20
56/56 [==============================] - 8s 135ms/step - loss: 14.5262 - accuracy: 0.4490 - val_loss: 12.2466 - val_accuracy: 0.3947
Epoch 4/20
56/56 [==============================] - 8s 136ms/step - loss: 10.8890 - accuracy: 0.4263 - val_loss: 7.0573 - val_accuracy: 0.3947
Epoch 5/20
56/56 [==============================] - 8s 137ms/step - loss: 7.2323 - accuracy: 0.3923 - val_loss: 22.6156 - val_accuracy: 0.3947
Epoch 6/20
56/56 [==============================] - 8s 138ms/step - loss: 13.4790 - accuracy: 0.4104 - val_loss: 5.2645 - val_accuracy: 0.3947
Epoch 7/20
56/56 [==============================] - 8s 138ms/step - loss: 8.2944 - accuracy: 0.4150 - val_loss: 24.8858 - v